### Required Libraries

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from enum import Enum
import re
import warnings

# Suppress warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore', category=UserWarning)

### Rule-based Parser

In [2]:
class TransactionType(Enum):
    INCOME = "income"
    EXPENSE = "expense"
    TRANSFER = "transfer"
    UNKNOWN = "unknown"

class Transaction:
    def __init__(self):
        self.user = None
        self.timestamp = datetime.now().isoformat()
        self.transaction_type = TransactionType.UNKNOWN
        self.amount = None
        self.account_number = None
        self.from_account = None
        self.to_account = None
        self.balance = None
        self.category = None
    
    def to_dict(self):
        # Get transaction type as string
        transaction_type_str = self.transaction_type.value if isinstance(self.transaction_type, TransactionType) else str(self.transaction_type)
        
        # Determine the final category based on transaction_type if category is not set
        final_category = self.category
        if not final_category and self.transaction_type != TransactionType.UNKNOWN:
            if self.transaction_type == TransactionType.INCOME:
                final_category = "other"
            elif self.transaction_type == TransactionType.EXPENSE:
                final_category = "other"
            elif self.transaction_type == TransactionType.TRANSFER:
                final_category = "general"
        
        return {
            "user": self.user,
            "timestamp": self.timestamp,
            "transaction_type": transaction_type_str,
            "amount": self.amount,
            "account_number": self.account_number,
            "from_account": self.from_account,
            "to_account": self.to_account,
            "balance": self.balance,
            "category": final_category,
        }

class RuleBasedParser:
    def __init__(self):
        # Keywords that typically indicate transaction categories - simplified without prefixes
        self.category_keywords = {
            # Income categories
            "salary": "salary",
            "payroll": "salary",
            "dividend": "investment",
            "interest": "interest",
            "refund": "refund",
            "cashback": "cashback",
            
            # Expense categories
            "grocery": "groceries",
            "supermarket": "groceries",
            "restaurant": "dining",
            "food": "dining",
            "cafe": "dining",
            "cinema": "entertainment",
            "movie": "entertainment",
            "uber": "transportation",
            "taxi": "transportation",
            "bus": "transportation",
            "metro": "transportation",
            "train": "transportation",
            "hospital": "healthcare",
            "doctor": "healthcare",
            "pharmacy": "healthcare",
            "medicine": "healthcare",
            "utility": "utilities",
            "electricity": "utilities",
            "water": "utilities",
            "internet": "utilities",
            "broadband": "utilities",
            "phone": "utilities",
            "mobile": "utilities",
            "rent": "housing",
            "mortgage": "housing",
            "insurance": "insurance",
            "repair": "maintenance",
            "maintenance": "maintenance",
            "education": "education",
            "tuition": "education",
            "book": "education",
            "clothing": "shopping",
            "shopping": "shopping",
            "entertainment": "entertainment",
            "gym": "fitness",
            "fitness": "fitness",
            "travel": "travel",
            "hotel": "travel",
            "flight": "travel",
            "subscription": "subscription",
            "donation": "charity",
            "charity": "charity",
            
            # Transfer categories
            "transfer": "general",
            "payment": "payment",
            "sent": "sent",
            "received": "received",
        }
        
        # Configure patterns to extract transaction details
        # Add more patterns to handle various SMS formats
        self.patterns = [
            # UPI debit pattern (highest priority)
            {
                "regex": r"(?:UPI user A\/C|A\/C)\s+([A-Za-z0-9]+)\s+debited by\s+([\d,.]+)(?:\.0)?\s+on date\s+(\d{2}[A-Za-z]{3}\d{2})\s+trf to\s+(.*?)\s+Refno\s+(\d+)",
                "type": TransactionType.EXPENSE,
                "extract": lambda m: {
                    "account": m.group(1),
                    "amount": m.group(2),
                    "date": m.group(3),
                    "currency": "INR"  # Assuming INR for UPI transactions
                }
            },
            # UPI credit pattern
            {
                "regex": r"(?:UPI user A\/C|A\/C)\s+([A-Za-z0-9]+)\s+credited by\s+([\d,.]+)(?:\.0)?\s+on date\s+(\d{2}[A-Za-z]{3}\d{2})\s+rcvd from\s+(.*?)\s+Refno\s+(\d+)",
                "type": TransactionType.INCOME,
                "extract": lambda m: {
                    "account": m.group(1),
                    "amount": m.group(2),
                    "date": m.group(3),
                    "currency": "INR"
                }
            },
            # Standard credit pattern
            {
                "regex": r"(?:Dear\s+Customer,\s+)?(\w+)?\s*([\d,.]+)\s+(?:was )?credited to your (?:account|a\/c)\s+[*]+(\d+)",
                "type": TransactionType.INCOME,
                "extract": lambda m: {
                    "amount": m.group(2), 
                    "account": m.group(3)
                }
            },
            # Standard debit pattern
            {
                "regex": r"(?:Dear\s+Customer,\s+)?(\w+)?\s*([\d,.]+)\s+(?:was )?debited from your (?:account|a\/c)\s+[*]+(\d+)",
                "type": TransactionType.EXPENSE,
                "extract": lambda m: {
                    "amount": m.group(2), 
                    "account": m.group(3)
                }
            },
            # HDFC debit pattern
            {
                "regex": r"Alert:\s+Rs\.([\d,.]+)\s+debited from a\/c\s+[*]+(\d+)\s+on\s+(\d+-\d+-\d+)(?:\s+to\s+(.*?))?(?:\.\s+Avlbl\s+Bal\s+Rs\.([\d,.]+))?",
                "type": TransactionType.EXPENSE,
                "extract": lambda m: {
                    "amount": m.group(1),
                    "account": m.group(2),
                    "date": m.group(3),
                    "balance": m.group(5) if len(m.groups()) > 4 and m.group(5) else None
                }
            },
            # HDFC credit pattern
            {
                "regex": r"Alert:\s+Rs\.([\d,.]+)\s+credited to a\/c\s+[*]+(\d+)\s+on\s+(\d+-\d+-\d+)(?:\s+from\s+(.*?))?(?:\.\s+Avlbl\s+Bal\s+Rs\.([\d,.]+))?",
                "type": TransactionType.INCOME,
                "extract": lambda m: {
                    "amount": m.group(1),
                    "account": m.group(2),
                    "date": m.group(3),
                    "balance": m.group(5) if len(m.groups()) > 4 and m.group(5) else None
                }
            },
            # SBI debit pattern
            {
                "regex": r"Your\s+(?:a\/c|account)\s+[*]+(\d+)\s+(?:is )?debited\s+(?:by )?(?:Rs.|INR)?\s*([\d,.]+)",
                "type": TransactionType.EXPENSE,
                "extract": lambda m: {
                    "amount": m.group(2),
                    "account": m.group(1)
                }
            },
            # SBI credit pattern
            {
                "regex": r"Your\s+(?:a\/c|account)\s+[*]+(\d+)\s+(?:is )?credited\s+(?:by )?(?:rs.|INR)?\s*([\d,.]+)",
                "type": TransactionType.INCOME,
                "extract": lambda m: {
                    "amount": m.group(2),
                    "account": m.group(1)
                }
            },
            # General debit pattern with amount at beginning
            {
                "regex": r"(?:rs|inr)?\s*([\d,.]+)\s+(?:has been |was )?debited\s+from\s+(?:your\s+)?(?:a\/c|account)\s+(?:[*]+)?(\d+)",
                "type": TransactionType.EXPENSE,
                "extract": lambda m: {
                    "amount": m.group(1),
                    "account": m.group(2)
                }
            },
            # General credit pattern with amount at beginning
            {
                "regex": r"(?:rs|inr)?\s*([\d,.]+)\s+(?:has been |was )?credited\s+to\s+(?:your\s+)?(?:a\/c|account)\s+(?:[*]+)?(\d+)",
                "type": TransactionType.INCOME,
                "extract": lambda m: {
                    "amount": m.group(1),
                    "account": m.group(2)
                }
            },
            # Transfer pattern
            {
                "regex": r"transfer[ed]*\s+(\w+)?\s*([\d,.]+)\s+from\s+[*]+(\d+)\s+to\s+[*]+(\d+)",
                "type": TransactionType.TRANSFER,
                "extract": lambda m: {
                    "amount": m.group(2), 
                    "from_account": m.group(3), 
                    "to_account": m.group(4)
                }
            },
            # Card transaction pattern
            {
                "regex": r"(?:INR|Rs\.?)\s*([\d,.]+)\s+spent on (?:your\s+)?(?:card|credit card) [*]+(\d+) at\s+(.*?)(?:\s+on\s+(\d+-\d+-\d+))?",
                "type": TransactionType.EXPENSE,
                "extract": lambda m: {
                    "amount": m.group(1),
                    "account": m.group(2)
                }
            },
            # ATM withdrawal pattern
            {
                "regex": r"(?:INR|Rs\.?)\s*([\d,.]+)\s+withdrawn from\s+(?:ATM|atm)(?:\s+at\s+(.*?))?(?:\s+on\s+(\d+-\d+-\d+))?",
                "type": TransactionType.EXPENSE,
                "extract": lambda m: {
                    "amount": m.group(1),
                    "category": "cash_withdrawal"
                }
            },
        ]
    
    def categorize_by_keywords(self, text):
        """Categorize text using simple keyword matching"""
        if not text:
            return None
            
        text_lower = text.lower()
        
        # Direct keyword matching
        for keyword, category in self.category_keywords.items():
            if keyword.lower() in text_lower:
                return category
        
        return None
    
    def extract_user_from_message(self, message):
        """Extract user information from the message if available"""
        user_pattern = re.search(r"User:\s*(\w+)", message)
        if user_pattern:
            return user_pattern.group(1)
        
        # Try to extract from message sender
        sender_pattern = re.search(r"^([A-Za-z0-9-]+):", message)
        if sender_pattern:
            return sender_pattern.group(1)
            
        return None
    
    def parse_date(self, date_str):
        """Parse date in various formats"""
        try:
            # Handle formats like 07Apr25
            if re.match(r"\d{2}[A-Za-z]{3}\d{2}", date_str):
                return datetime.strptime(date_str, "%d%b%y").date()
            # Handle formats like 07-04-2023
            elif re.match(r"\d{1,2}-\d{1,2}-\d{2,4}", date_str):
                # Try different formats
                for fmt in ["%d-%m-%Y", "%d-%m-%y", "%m-%d-%Y", "%m-%d-%y"]:
                    try:
                        return datetime.strptime(date_str, fmt).date()
                    except ValueError:
                        continue
            return None
        except:
            return None
    
    def parse_notification(self, raw_input):
        """Parse raw notification and extract transaction details using regex patterns"""
        transaction = Transaction()
        
        # Extract user
        transaction.user = self.extract_user_from_message(raw_input)
        
        # Extract balance if available
        balance_match = re.search(r"available (?:account )?balance(?:\s+is)?\s+(\w+)?\s*([\d,.]+)", raw_input, re.IGNORECASE)
        if balance_match:
            transaction.balance = float(balance_match.group(2).replace(',', ''))
        
        # Try all patterns in order of priority
        for pattern in self.patterns:
            match = re.search(pattern["regex"], raw_input, re.IGNORECASE)
            if match:
                transaction.transaction_type = pattern["type"]
                data = pattern["extract"](match)
                
                if "amount" in data:
                    transaction.amount = float(data["amount"].replace(',', ''))
                
                if "account" in data:
                    transaction.account_number = data["account"]
                    if pattern["type"] == TransactionType.INCOME:
                        transaction.to_account = data["account"]
                    elif pattern["type"] == TransactionType.EXPENSE:
                        transaction.from_account = data["account"]
                
                if "from_account" in data:
                    transaction.from_account = data["from_account"]
                
                if "to_account" in data:
                    transaction.to_account = data["to_account"]
                
                if "date" in data and data["date"]:
                    parsed_date = self.parse_date(data["date"])
                    if parsed_date:
                        transaction.timestamp = datetime.combine(
                            parsed_date, 
                            datetime.now().time()
                        ).isoformat()
                
                if "balance" in data and data["balance"]:
                    transaction.balance = float(data["balance"].replace(',', ''))
                
                if "category" in data:
                    transaction.category = data["category"]
                
                break
        
        # If no pattern matched, try generic extraction
        if transaction.transaction_type == TransactionType.UNKNOWN:
            # Try to detect transaction type based on keywords
            if re.search(r'\b(credit(?:ed)?|receiv(?:ed|e)|deposit(?:ed)?)\b', raw_input, re.IGNORECASE):
                transaction.transaction_type = TransactionType.INCOME
            elif re.search(r'\b(debit(?:ed)?|withdraw(?:n|al)|spent|payment|purchase|paid)\b', raw_input, re.IGNORECASE):
                transaction.transaction_type = TransactionType.EXPENSE
            elif re.search(r'\b(transfer(?:red)?|sent|moved)\b', raw_input, re.IGNORECASE):
                transaction.transaction_type = TransactionType.TRANSFER
            
            # Extract amount if not already done
            if transaction.amount is None:
                amount_match = re.search(r'(?:RS|Rs|INR|USD|EUR|GBP|AED)\.?\s*([\d,]+\.?\d*)', raw_input, re.IGNORECASE)
                if amount_match:
                    try:
                        transaction.amount = float(amount_match.group(1).replace(',', ''))
                    except ValueError:
                        pass
                else:
                    # Try just the number
                    amount_match = re.search(r'([\d,]+\.?\d*)', raw_input)
                    if amount_match:
                        try:
                            transaction.amount = float(amount_match.group(1).replace(',', ''))
                        except ValueError:
                            pass
            
            # Try to extract account number
            account_match = re.search(r'(?:a\/c|account)\s+[*x]+(\d+)', raw_input, re.IGNORECASE)
            if account_match:
                transaction.account_number = account_match.group(1)
        
        # Try to categorize based on message content
        if not transaction.category:
            transaction.category = self.categorize_by_keywords(raw_input)
            
        # If still no category but we know the transaction type
        if not transaction.category and transaction.transaction_type != TransactionType.UNKNOWN:
            if transaction.transaction_type == TransactionType.INCOME:
                transaction.category = "other"
            elif transaction.transaction_type == TransactionType.EXPENSE:
                transaction.category = "other"
            elif transaction.transaction_type == TransactionType.TRANSFER:
                transaction.category = "general"
        
        return transaction

def categorize_bank_sms(df):
    """Process and categorize bank SMS data from DataFrame"""
    # Create parser
    parser = RuleBasedParser()
    
    # Create lists to store extracted data
    transactions = []
    
    # Process each SMS
    total_sms = len(df)
    print(f"Processing {total_sms} SMS messages...")
    
    for idx, row in df.iterrows():        
        # Prepare input with user information
        input_text = f"User: {row['Cardholders']}\nSMS: {row['SMS']}"
        
        # Parse notification
        transaction = parser.parse_notification(input_text)
        
        # Add to list
        transactions.append(transaction.to_dict())
    
    # Convert to DataFrame
    transactions_df = pd.DataFrame(transactions)
    
    # Clean up and convert data types
    if 'amount' in transactions_df.columns:
        transactions_df['amount'] = pd.to_numeric(transactions_df['amount'], errors='coerce')
    
    if 'balance' in transactions_df.columns:
        transactions_df['balance'] = pd.to_numeric(transactions_df['balance'], errors='coerce')
    
    if 'timestamp' in transactions_df.columns:
        transactions_df['timestamp'] = pd.to_datetime(transactions_df['timestamp'], errors='coerce')
    
    return transactions_df

### Testing with dataset

In [5]:
file_path = 'Prepared bank transactions SMS dataset .xlsx'

# Load data
try:
    df = pd.read_excel(file_path)
    required_columns = ['Cardholders', 'SMS']
    
    # Ensure required columns exist
    if not all(col in df.columns for col in required_columns):
        # Try to identify appropriate columns
        if len(df.columns) >= 2:
            # Rename first two columns to required names
            df = df.iloc[:, :2]
            df.columns = required_columns
            print("Automatically renamed columns to 'Cardholders' and 'SMS'")
        else:
            print("Data format is incorrect. Please ensure there are at least two columns.")
    
    # Keep only the required columns
    df = df[required_columns]
    
    print("Shape of the dataset:", df.shape)
    print("\nFirst 5 rows:")
    print(df.head())
    
    unique_users = df['Cardholders'].nunique()
    print(f"\nNumber of unique users: {unique_users}")
    
    # Process transactions
    transactions_df = categorize_bank_sms(df)
    
    # Display and save results
    print("Transactions CSV:")
    print(transactions_df.head())  # Show first few rows
    transactions_df.to_csv('transactions.csv', index=False)

    # Additional statistics
    print("\nTransaction Type Distribution:")
    if 'transaction_type' in transactions_df.columns:
        type_counts = transactions_df['transaction_type'].value_counts()
        print(type_counts)
    
    # Statistics on amount by transaction type
    if 'amount' in transactions_df.columns and 'transaction_type' in transactions_df.columns:
        print("\nAmount Statistics by Transaction Type:")
        transaction_types = transactions_df['transaction_type'].unique()
        for t_type in transaction_types:
            mask = transactions_df['transaction_type'] == t_type
            if mask.sum() > 0:
                amount_stats = transactions_df.loc[mask, 'amount'].describe()
                print(f"\n{t_type.capitalize()}:")
                print(f"  Count: {amount_stats['count']:.0f}")
                print(f"  Average: {amount_stats['mean']:.2f}")
                print(f"  Min: {amount_stats['min']:.2f}")
                print(f"  Max: {amount_stats['max']:.2f}")

except Exception as e:
    print(f"Error processing file: {e}")

Shape of the dataset: (1894, 2)

First 5 rows:
  Cardholders                                                SMS
0       user1         Account *****535 has been created for you.
1       user1  Dear Customer, thank you for opening a new AED...
2       user1  Dear Customer, thank you for requesting a new ...
3       user1  Dear Customer, AED 25806.50 was credited to yo...
4       user1  Dear Customer, AED 12800.00 was debited from y...

Number of unique users: 1
Processing 1894 SMS messages...
Transactions CSV:
    user                  timestamp transaction_type   amount account_number  \
0  user1 2025-05-11 10:26:58.225625          unknown      1.0            535   
1  user1 2025-05-11 10:26:58.226547          unknown      1.0           None   
2  user1 2025-05-11 10:26:58.226547          unknown      1.0           None   
3  user1 2025-05-11 10:26:58.226547           income  25806.5           0535   
4  user1 2025-05-11 10:26:58.226547          expense  12800.0           0535   

  fro

### Testing with manual input

In [6]:
def test_sms(sms_text, user_name="TestUser"):
    """
    Test a single SMS message with the transaction parser
    
    Args:
        sms_text (str): The SMS text to parse
        user_name (str): Optional cardholder name (default: 'TestUser')
        
    Returns:
        dict: The parsed transaction details
    """
    # Create parser
    parser = RuleBasedParser()  # Changed from NotificationParser to RuleBasedParser
    
    # Prepare input with user information
    input_text = f"User: {user_name}\nSMS: {sms_text}"
    
    # Parse notification
    transaction = parser.parse_notification(input_text)
    
    # Return dictionary representation
    return transaction.to_dict()


sample_sms = "Payment successfully made to TOKOPEDIA. An amount of Rp997 has been made from your GoPay."
result = test_sms(sample_sms)

print("\nResult:")
for key, value in result.items():
    print(f"  {key}: {value}")


Result:
  user: TestUser
  timestamp: 2025-05-11T10:27:02.600667
  transaction_type: expense
  amount: 997.0
  account_number: None
  from_account: None
  to_account: None
  balance: None
  category: payment
